In [ ]:
#feedback: reproducibility를 위해 randomness 통제를 위한 다음의 코드 추가하면 좋을 것 같습니다. (해당 시드는 고정 후, 아래 TrainingArguments의 seed 만 0-9까지 바꾸는 식으로 실험 진행하시면 될 것 같습니다.)
import numpy as np
import random
import torch
np.random.seed(0)
random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)

If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 10.0 MB/s eta 0:00:00


In [ ]:
# feedback: notebook_login()은 현재로서는 필요하지 않은 것 같습니다.
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install transformers==4.28.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.5 MB/s eta 0:00:00


In [ ]:
import transformers
print(transformers.__version__)

4.28.0


In [ ]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"] 

SST-2 (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
`label (class label)에서 negative:0 positive:1`

In [ ]:
task = "sst2"
model_checkpoint = "bert-base-uncased"
batch_size = 10

# Loading the dataset

In [ ]:
from datasets import load_dataset, load_metric, concatenate_datasets

In [ ]:

# actual_task = "sst2"
# dataset = load_dataset("glue", actual_task) #데이터셋 불러오기
# metric = load_metric('glue', actual_task)
dataset = load_dataset("ag_news")
metric = load_metric('accuracy')

# train과 validation 데이터셋 합치기
train_data = dataset['train']
# val_data = dataset['validation']
test_data = dataset['test']
combined_data = train_data

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-9-b4bcbfb8de27>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('accuracy')


In [ ]:
dataset #train, dev, test 데이터셋

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [ ]:
combined_data #train 과 val 을 합친 데이터셋, *주의할점* train, dev를 합쳐서 class마다 n개를 뽑아야함

Dataset({
    features: ['text', 'label'],
    num_rows: 120000
})

In [ ]:
#class(label) 마다 n개씩 추출하기 위해서 class마다 수 확인하기, positive:1, negative:0
num0 = 0
num1 = 0
num2 = 0
num3 = 0

for i in range(0, len(combined_data)):
  if combined_data[i]['label'] == 0:
    num0 += 1
  if combined_data[i]['label'] == 1:
    num1 += 1
  if combined_data[i]['label'] == 2:
    num2 += 1
  if combined_data[i]['label'] == 3:
    num3 += 1

print(num0, num1, num2, num3)


30000 30000 30000 30000


In [ ]:
# Dataset.filter() 함수를 사용하여 레이블이 positive인 데이터와 negative인 데이터를 각각 필터링
num0_data = combined_data.filter(lambda example: example['label'] == 0)
num1_data = combined_data.filter(lambda example: example['label'] == 1)
num2_data = combined_data.filter(lambda example: example['label'] == 2)
num3_data = combined_data.filter(lambda example: example['label'] == 3)

Filter:   0%|          | 0/120000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/120000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/120000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/120000 [00:00<?, ? examples/s]

In [ ]:
#다음으로, positive_data와 negative_data에서 각각 무작위로 10개씩 데이터를 추출. 
#이를 위해서는 Dataset.shuffle() 함수를 사용
#여기서 shuffle() 함수는 데이터를 무작위로 섞는 역할을 하고, select() 함수는 추출할 데이터의 인덱스를 지정.
num0_data = num0_data.shuffle(seed=2).select(range(50))
num1_data = num1_data.shuffle(seed=2).select(range(50))
num2_data = num2_data.shuffle(seed=2).select(range(50))
num3_data = num3_data.shuffle(seed=2).select(range(50))
 

#이 코드를 실행하면, positive_data와 negative_data에서 각각 무작위로 10개씩 데이터가 추출.
print(num0_data)

Dataset({
    features: ['text', 'label'],
    num_rows: 50
})


In [ ]:
#positive_data와 negative_data를 합친 새로운 데이터셋
sampled_data = concatenate_datasets([num0_data, num1_data, num2_data, num3_data])
print(sampled_data)


Dataset({
    features: ['text', 'label'],
    num_rows: 200
})


In [ ]:
print(sampled_data['text'])

['Confusion as Yemeni Faces U.S. Military Tribunal (Reuters) Reuters - A Yemeni\\accused of guarding Osama bin Laden with explosive belts seemed\\to acknowledge his al Qaeda membership on Thursday before a\\U.S. military tribunal hearing fraught with translation\\problems.', 'KILLING ZONE Iraq has a population of about 20 million. According to Interim Prime Minister Ayad Allawi, most are living in relative peace and safety.', 'Norway Searches for Treasured Masterpieces OSLO, Norway - Norwegian police said Monday that they were working on several tips in their search for two Edvard Munch masterpieces, including a version of "The Scream," after a bold daytime theft from an Oslo museum in front of stunned visitors.    The paintings were stolen from Oslo\'s Munch Museum on Sunday by masked, armed thieves, who pulled them from the walls as visitors and staff watched...', 'EU weighs Iranian nuclear reply EU countries analyse an Iranian response to EU pressure for it to halt plans to enrich u

In [ ]:
# sampled_data와 test_data를 따로 분리해서 새로운 데이터셋을 만들기 위해 DatasetDict를 사용
from sklearn.model_selection import train_test_split
from datasets import DatasetDict
# train_test_split() 함수는 데이터를 무작위로 섞어주고, 지정한 test_size 비율에 따라 데이터를 분할
# sampled_data를 train과 validation(dev) 데이터셋으로 나누기 위해 train_test_split() 함수를 사용
train_dev = sampled_data.train_test_split(shuffle = True, seed = 0, test_size=0.1)

#train_dev와 test_data를 합쳐서 새로운 DatasetDict을 만들기
#새로운 new_dataset은 train, validation, test의 key를 가지고 
#각각 해당하는 데이터셋을 value로 가지는 DatasetDict
new_dataset = DatasetDict({'train': train_dev['train'], 
                           'validation': train_dev['test'],
                           'test': test_data})

print(new_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 180
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 20
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
#Hugging Face의 Datasets 라이브러리를 사용하여 데이터셋에서 
#무작위로 num_examples개의 데이터를 선택하고, Pandas DataFrame을 사용하여 이를 표시하는 함수
import datasets
# import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = [] # 리스트를 초기화
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick) #선택된 인덱스를 picks 리스트에 추가합니다.

    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
#sample_data = show_random_elements(combined_data) #합쳐진 데이터에서 10개를 무작위로 추출한 데이터
#여기서 train-val 비율을 나눠 학습 시킨다.

The metric is an instance of datasets.Metric:

In [ ]:
metric

Metric(name: "accuracy", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = datasets.load_metric("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
        {'accuracy': 0.5}

   

In [ ]:
# import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.46875}

# Preprocessing the data

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True, )

In [ ]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [ ]:
task_to_keys[task] = ('text', None)

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.


In [ ]:
def preprocess_function(examples):
    if sentence2_key is None:
        # return tokenizer(examples[sentence1_key], truncation=True)
        return tokenizer(examples[sentence1_key], padding='max_length', max_length=80 , truncation=True) #feedback:  padding과 max_length 설정을 하는 것이 좋을 것 같습니다. padding 옵션은 그대로 두시고, dataset 마다 max_legnth는 그에 맞게 수정하시면 좋을 것 같습니다.
    # return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], padding='max_length', max_length=80 , truncation=True)

In [ ]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 2813, 2358, 1012, 6468, 15020, 2067, 2046, 1996, 2304, 1006, 26665, 1007, 26665, 1011, 2460, 1011, 19041, 1010, 2813, 2395, 1005, 1055, 1040, 11101, 2989, 1032, 2316, 1997, 11087, 1011, 22330, 8713, 2015, 1010, 2024, 3773, 2665, 2153, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 18431, 2571, 3504, 2646, 3293, 13395, 1006, 26665, 1007, 26665, 1011, 2797, 5211, 3813, 18431, 2571, 2177, 1010, 1032, 2029, 2038, 1037, 5891, 2005, 2437, 2092, 1011, 22313, 1998, 5681, 1032, 6801, 3248, 1999, 1996, 3639, 3068, 1010, 2038, 5168, 2872, 1032, 2049, 29475, 2006, 2178, 2112, 1997, 1996, 3006, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 3514, 1998, 4610, 6112, 15768, 1005, 17680, 1006, 26665, 1007, 26665, 1011, 23990, 13587, 7597, 4606, 15508, 1032, 2055, 1996, 4610, 1998, 1996, 17680, 2005, 16565, 2024, 3517, 2000, 1032, 6865, 2058, 19

In [ ]:
encoded_dataset = new_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 180
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7600
    })
})

# Fine-tuning the model

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
num_labels = 4
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
print(transformers.__version__)

4.28.0


# seed 0

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    # learning_rate=2e-5,
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # num_train_epochs=5,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # push_to_hub=True, #feedback: 해당 옵션을 활용하지 않으셔도 될 것 같습니다.
    seed = 0, # 0~
)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 180
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 20
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7600
    })
})

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.342465,0.500000
2,No log,1.221666,0.600000
3,No log,1.042260,0.800000
4,No log,0.822314,0.900000
5,No log,0.584814,0.950000
6,No log,0.442641,0.950000
7,No log,0.308882,0.950000
8,No log,0.237455,0.950000
9,No log,0.198056,0.950000
10,No log,0.164108,0.950000


KeyboardInterrupt: ignored

In [ ]:
trainer.evaluate()

In [ ]:
trainer.predict(encoded_dataset["test"])

# seed 1

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    # learning_rate=2e-5,
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # num_train_epochs=5,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # push_to_hub=True, #feedback: 해당 옵션을 활용하지 않으셔도 될 것 같습니다.
    seed = 1, # 0~
)

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.predict(encoded_dataset["test"])

# seed 2

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    # learning_rate=2e-5,
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # num_train_epochs=5,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # push_to_hub=True, #feedback: 해당 옵션을 활용하지 않으셔도 될 것 같습니다.
    seed = 2, # 0~
)

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.predict(encoded_dataset["test"])

# seed 3

In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    # learning_rate=2e-5,
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # num_train_epochs=5,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # push_to_hub=True, #feedback: 해당 옵션을 활용하지 않으셔도 될 것 같습니다.
    seed = 3, # 0~
)

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.predict(encoded_dataset["test"])

# seed 4



In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    # learning_rate=2e-5,
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # num_train_epochs=5,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # push_to_hub=True, #feedback: 해당 옵션을 활용하지 않으셔도 될 것 같습니다.
    seed = 4, # 0~
)

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.816192,0.716667
2,No log,0.773758,0.733333
3,No log,0.768834,0.733333
4,No log,0.772967,0.733333
5,No log,0.750558,0.738889
6,No log,0.745081,0.738889
7,No log,0.742635,0.722222
8,No log,0.740520,0.733333
9,No log,0.729762,0.733333
10,No log,0.726890,0.727778


TrainOutput(global_step=60, training_loss=0.06420243581136068, metrics={'train_runtime': 189.5112, 'train_samples_per_second': 3.166, 'train_steps_per_second': 0.317, 'total_flos': 24667104384000.0, 'train_loss': 0.06420243581136068, 'epoch': 30.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6897379755973816,
 'eval_accuracy': 0.7444444444444445,
 'eval_runtime': 0.9481,
 'eval_samples_per_second': 189.862,
 'eval_steps_per_second': 18.986,
 'epoch': 30.0}

In [ ]:
trainer.predict(encoded_dataset["test"])

PredictionOutput(predictions=array([[ 1.5599022 , -0.6885723 ,  0.01033592, -0.97693574],
       [ 0.5642462 ,  0.6762204 ,  0.14220841, -0.10685384],
       [ 0.0458206 ,  0.717955  ,  0.43055892,  0.3488017 ],
       ...,
       [-0.29501238,  2.477314  , -0.56514144, -0.769355  ],
       [ 0.3869379 ,  0.04115503,  0.40986887,  0.41731992],
       [-1.3309741 , -0.9213377 ,  1.9512147 ,  1.0480043 ]],
      dtype=float32), label_ids=array([2, 3, 3, ..., 1, 2, 2]), metrics={'test_loss': 0.7194573879241943, 'test_accuracy': 0.7344736842105263, 'test_runtime': 39.0628, 'test_samples_per_second': 194.559, 'test_steps_per_second': 19.456})

# seed 5



In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    # learning_rate=2e-5,
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # num_train_epochs=5,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # push_to_hub=True, #feedback: 해당 옵션을 활용하지 않으셔도 될 것 같습니다.
    seed = 5, # 0~
)

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.672794,0.744444
2,No log,0.703236,0.733333
3,No log,0.693894,0.727778
4,No log,0.670382,0.755556
5,No log,0.659074,0.766667
6,No log,0.654064,0.766667
7,No log,0.649996,0.761111
8,No log,0.651304,0.755556
9,No log,0.658811,0.755556
10,No log,0.662612,0.750000


TrainOutput(global_step=60, training_loss=0.024103927612304687, metrics={'train_runtime': 189.5236, 'train_samples_per_second': 3.166, 'train_steps_per_second': 0.317, 'total_flos': 24667104384000.0, 'train_loss': 0.024103927612304687, 'epoch': 30.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6377294063568115,
 'eval_accuracy': 0.7722222222222223,
 'eval_runtime': 0.9747,
 'eval_samples_per_second': 184.678,
 'eval_steps_per_second': 18.468,
 'epoch': 30.0}

In [ ]:
trainer.predict(encoded_dataset["test"])

PredictionOutput(predictions=array([[ 1.7157472 , -0.90985185,  0.01704115, -1.1690844 ],
       [ 0.35048124,  0.39519683,  0.38681537,  0.1700624 ],
       [-0.27853635,  0.62895304,  0.4964832 ,  0.75292677],
       ...,
       [-0.488288  ,  3.0995982 , -0.8703919 , -1.0228227 ],
       [ 0.27318367, -0.13540997,  0.48384315,  0.59954387],
       [-1.6619085 , -1.3166418 ,  2.1494875 ,  1.3424432 ]],
      dtype=float32), label_ids=array([2, 3, 3, ..., 1, 2, 2]), metrics={'test_loss': 0.6837801933288574, 'test_accuracy': 0.7553947368421052, 'test_runtime': 39.3709, 'test_samples_per_second': 193.036, 'test_steps_per_second': 19.304})

# seed 6



In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    # learning_rate=2e-5,
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # num_train_epochs=5,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # push_to_hub=True, #feedback: 해당 옵션을 활용하지 않으셔도 될 것 같습니다.
    seed = 6, # 0~
)

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.650285,0.777778
2,No log,0.683454,0.755556
3,No log,0.689833,0.750000
4,No log,0.685152,0.761111
5,No log,0.661502,0.772222
6,No log,0.638677,0.783333
7,No log,0.632523,0.788889
8,No log,0.639680,0.783333
9,No log,0.648779,0.783333
10,No log,0.668861,0.783333


TrainOutput(global_step=60, training_loss=0.01144472857316335, metrics={'train_runtime': 189.9759, 'train_samples_per_second': 3.158, 'train_steps_per_second': 0.316, 'total_flos': 24667104384000.0, 'train_loss': 0.01144472857316335, 'epoch': 30.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6325225830078125,
 'eval_accuracy': 0.7888888888888889,
 'eval_runtime': 0.9331,
 'eval_samples_per_second': 192.911,
 'eval_steps_per_second': 19.291,
 'epoch': 30.0}

In [ ]:
trainer.predict(encoded_dataset["test"])

PredictionOutput(predictions=array([[ 2.0918255 , -1.0694516 ,  0.14774024, -1.5496798 ],
       [ 0.1542484 ,  0.3077717 ,  0.787312  ,  0.12084325],
       [-0.53664523,  0.56085014,  0.6163641 ,  1.0041256 ],
       ...,
       [-0.6971493 ,  3.5281694 , -0.981065  , -1.1641996 ],
       [ 0.27215976, -0.38234764,  0.84865284,  0.63823044],
       [-1.8232596 , -1.3456271 ,  2.6957378 ,  1.4052864 ]],
      dtype=float32), label_ids=array([2, 3, 3, ..., 1, 2, 2]), metrics={'test_loss': 0.6925299167633057, 'test_accuracy': 0.761578947368421, 'test_runtime': 39.0119, 'test_samples_per_second': 194.812, 'test_steps_per_second': 19.481})

# seed 7



In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    # learning_rate=2e-5,
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # num_train_epochs=5,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # push_to_hub=True, #feedback: 해당 옵션을 활용하지 않으셔도 될 것 같습니다.
    seed = 7, # 0~
)

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.650553,0.777778
2,No log,0.653653,0.783333
3,No log,0.670542,0.783333
4,No log,0.691049,0.777778
5,No log,0.692418,0.777778
6,No log,0.684964,0.772222
7,No log,0.674611,0.777778
8,No log,0.670411,0.766667
9,No log,0.665390,0.777778
10,No log,0.668916,0.777778


TrainOutput(global_step=60, training_loss=0.007928314805030822, metrics={'train_runtime': 188.3476, 'train_samples_per_second': 3.186, 'train_steps_per_second': 0.319, 'total_flos': 24667104384000.0, 'train_loss': 0.007928314805030822, 'epoch': 30.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6536533236503601,
 'eval_accuracy': 0.7833333333333333,
 'eval_runtime': 0.9194,
 'eval_samples_per_second': 195.77,
 'eval_steps_per_second': 19.577,
 'epoch': 30.0}

In [ ]:
trainer.predict(encoded_dataset["test"])

PredictionOutput(predictions=array([[ 1.8068469 , -0.98335713, -0.08990058, -1.2634771 ],
       [ 0.05501479,  0.42129093,  0.2898972 ,  0.36715   ],
       [-0.67876333,  0.79502255,  0.25042084,  1.1043274 ],
       ...,
       [-0.9194117 ,  3.5609052 , -1.0461102 , -1.0081556 ],
       [ 0.13311377, -0.30681   ,  0.5948301 ,  0.76140547],
       [-1.8050768 , -1.0209492 ,  2.5204985 ,  1.6569802 ]],
      dtype=float32), label_ids=array([2, 3, 3, ..., 1, 2, 2]), metrics={'test_loss': 0.7061087489128113, 'test_accuracy': 0.76, 'test_runtime': 37.4966, 'test_samples_per_second': 202.685, 'test_steps_per_second': 20.269})

# seed 8



In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    # learning_rate=2e-5,
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # num_train_epochs=5,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # push_to_hub=True, #feedback: 해당 옵션을 활용하지 않으셔도 될 것 같습니다.
    seed = 8, # 0~
)

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.687215,0.755556
2,No log,0.695789,0.761111
3,No log,0.683067,0.783333
4,No log,0.663813,0.783333
5,No log,0.654644,0.788889
6,No log,0.648094,0.788889
7,No log,0.649046,0.788889
8,No log,0.659712,0.788889
9,No log,0.678927,0.783333
10,No log,0.696397,0.783333


KeyboardInterrupt: ignored

In [ ]:
trainer.evaluate()

In [ ]:
trainer.predict(encoded_dataset["test"])

# seed 9



In [ ]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    # learning_rate=2e-5,
    learning_rate=1e-5, #feedback: learning rate은 이전 논문들의 hyperparamter를 참고하여 1e-5로 고정하여 실험하는 것이 좋을 것 같습니다.
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # num_train_epochs=5,
    num_train_epochs=30, #feedback: epoch은 이전 논문의 hyperparamter를 참고하여 30으로 고정하여 실험하는 것이 좋을 것 같습니다.
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    # push_to_hub=True, #feedback: 해당 옵션을 활용하지 않으셔도 될 것 같습니다.
    seed = 9, # 0~
)

In [ ]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.predict(encoded_dataset["test"])